In [7]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output
import numpy as np
import torch
import sys
import matplotlib.pyplot as plt
import torch.nn as nn
import traceback
import pickle
import random
import torch.nn.functional as F
import os
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from sentence_transformers import SentenceTransformer
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from deepctr_torch.inputs import SparseFeat, VarLenSparseFeat, get_feature_names
from deepctr_torch.models.din import DIN
from deepctr_torch.models import DeepFM
import flwr as fl
from collections import OrderedDict
from typing import List, Tuple
from flwr.common import (
    Code,
    EvaluateIns,
    EvaluateRes,
    FitIns,
    FitRes,
    GetParametersIns,
    GetParametersRes,
    Status,
    ndarrays_to_parameters,
    parameters_to_ndarrays,
)

np.random.seed(42)

In [8]:

def get_data(df):
    data = df.copy()
    sparse_features = ["movieId", "userId",
                       "gender", "age", "occupation", "zipCode"]
    target = ['rating']
    for feat in sparse_features:
        lbe = LabelEncoder()
        data[feat] = lbe.fit_transform(data[feat])
    # data['rating'] = data['rating'].apply(lambda x: 1 if x > 3 else 0)

    # 2.count #unique features for each sparse field
    fixlen_feature_columns = [SparseFeat(feat, data[feat].nunique(), embedding_dim=4)
                              for feat in sparse_features]

    linear_feature_columns = fixlen_feature_columns
    dnn_feature_columns = fixlen_feature_columns

    feature_names = get_feature_names(
        linear_feature_columns + dnn_feature_columns)

    return data, feature_names,


def show_chart(df):
    def plot_histogram(bins):
        plt.figure(figsize=(8, 6))
        plt.hist(df['rating'], bins=bins, color='skyblue', edgecolor='black')
        plt.title('Rating Distribution')
        plt.xlabel('Rating')
        plt.ylabel('Count')
        plt.show()

    # 创建滑块小部件
    bins_slider = widgets.IntSlider(
        value=10, min=1, max=20, step=1, description='Bins:')
    output = widgets.Output()

    # 柱状图交互
    def interactive_plot(change):
        with output:
            clear_output()
            plot_histogram(bins_slider.value)

    bins_slider.observe(interactive_plot, names='value')
    interactive_plot(None)

    # 显示滑块和柱状图
    display(bins_slider)
    display(output)


def show_movie_list(df):
    rows_per_page = 10
    # 创建两个按钮，一个用于下一页，一个用于上一页
    next_button = widgets.Button(description='下一页')
    prev_button = widgets.Button(description='上一页')
    page_label = widgets.Label()
    # 创建数据显示表格
    output = widgets.Output()
    current_page = 1

    def show_table(page_number):
        with output:
            clear_output()  # 清除上一页的内容
            start_idx = (page_number - 1) * rows_per_page
            end_idx = page_number * rows_per_page
            display(df.iloc[start_idx:end_idx])
            page_label.value = f'页数: {page_number}'

    show_table(current_page)

    def next_page(b):
        nonlocal current_page
        current_page += 1
        if current_page > len(df) // rows_per_page:
            current_page = len(df) // rows_per_page
        show_table(current_page)

    def prev_page(b):
        nonlocal current_page
        current_page -= 1
        if current_page < 1:
            current_page = 1
        show_table(current_page)

    next_button.on_click(next_page)
    prev_button.on_click(prev_page)
    # 显示表格、按钮和标签
    display(output)
    display(widgets.HBox([prev_button, next_button, page_label]))


def get_all_rating_one_user(model, movie_df, user_df, user_id):
    # 获取用户信息
    user_info = user_df[user_df['userId'] == user_id]

    # 为每个电影添加相同的用户信息
    movie_df['userId'] = user_info['userId'].values[0]  # 设置用户ID为指定用户
    movie_df['gender'] = user_info['gender'].values[0]  # 设置性别为指定用户
    movie_df['age'] = user_info['age'].values[0]  # 设置年龄为指定用户
    movie_df['occupation'] = user_info['occupation'].values[0]  # 设置职业为指定用户
    movie_df['zipCode'] = user_info['zipCode'].values[0]  # 设置邮编为指定用户

    # 获取电影评分（这部分你的代码需要根据你的模型和数据来完成）
    # movie_df['rating'] = model.predict(movie_df)  # 你需要根据你的模型来预测评分

    # 仅保留需要的列
    sparse_features = ["movieId", "userId", "gender",
                       "age", "occupation", "zipCode"]
    data = movie_df[sparse_features]

    data, feature_names = get_data(
        data)
    model_input = {name: data[name] for name in feature_names}
    # pred_ans = model.predict(model_input, batch_size=256)
    np.random.seed(user_id)
    movie_df['rating'] = np.random.normal(loc=3, scale=1, size=len(movie_df))
    movie_df['rating'] = movie_df['rating'].clip(0, 5)
    movie_df.sort_values(by='rating', ascending=False, inplace=True)
    return movie_df

In [9]:
%matplotlib inline

feature_names = pickle.load(open('feature_names.pkl', 'rb'))
linear_feature_columns = pickle.load(open('linear_feature_columns.pkl', 'rb'))
dnn_feature_columns = pickle.load(open('dnn_feature_columns.pkl', 'rb'))
device = 'cpu'
model = DeepFM(linear_feature_columns, dnn_feature_columns, dnn_hidden_units=(512, 256, 128),
               task='regression', device=device)
model.compile("adam", "mse", metrics=['mse'], )
model.load_state_dict(torch.load('model.pkl'))
movie_df = pd.read_csv('ml-1m/movies.csv')
user_df = pd.read_csv('ml-1m/users.csv')

# 创建输入框和按钮
input_text = widgets.Text(placeholder='输入文本', description='文本框:')
execute_button = widgets.Button(description='执行')
# 创建输出小部件以显示结果
output = widgets.Output()

#user_id
rating_df = get_all_rating_one_user(model, movie_df, user_df, 111)
show_movie_list(rating_df)
show_chart(rating_df)




Output()

IntSlider(value=10, description='Bins:', max=20, min=1)

Output()